# Mini Project 2 – Normalized Sales Database (PostgreSQL)

This notebook connects to a **PostgreSQL normalized sales database** and runs
the analysis queries `ex1`–`ex11` that were originally written for SQLite,
now converted to **PostgreSQL syntax**.

Tables used:

- `region(regionid, region)`
- `country(countryid, country, regionid)`
- `customer(customerid, firstname, lastname, address, city, countryid)`
- `productcategory(productcategoryid, productcategory, productcategorydescription)`
- `product(productid, productname, productunitprice, productcategoryid)`
- `orderdetail(orderid, customerid, productid, orderdate, quantityordered)`

The schema + data are assumed to already exist in PostgreSQL (for example, on Render).
We only **query** the database here, not rebuild it.

In [4]:
import pandas as pd
from sqlalchemy import create_engine, text

# PostgreSQL / Render credentials (fill these from your Render DB)
DB_USER = "mini_project2_sales_data_user"
DB_PASSWORD = "obYsdJaiTfTjfxhyXG7yXvGRpbh7a8PT"
DB_HOST = "dpg-d4kuukili9vc73e0jmp0-a.virginia-postgres.render.com" 
DB_PORT = "5432"
DB_NAME = "mini_project2_sales_data"

conn_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(conn_str)

# quick sanity check
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1 AS test"))
    print("Connection OK:", list(result))


Connection OK: [(1,)]


In [5]:
def run_query(sql: str):
    """Run a SQL query on the PostgreSQL engine and return a DataFrame."""
    with engine.connect() as conn:
        df = pd.read_sql(text(sql), conn)
    return df

In [17]:
def ex1(conn, CustomerName):
    return f"""
    SELECT
        c.firstname || ' ' || c.lastname AS Name,
        p.productname AS ProductName,
        o.orderdate AS OrderDate,
        p.productunitprice AS ProductUnitPrice,
        o.quantityordered AS QuantityOrdered,
        ROUND((p.productunitprice * o.quantityordered)::numeric, 2) AS Total
    FROM orderdetail o
    JOIN customer c ON o.customerid = c.customerid
    JOIN product  p ON o.productid  = p.productid
    WHERE (c.firstname || ' ' || c.lastname) = '{CustomerName}'
    ORDER BY o.orderdate;
    """


def ex2(conn, CustomerName):
    return f"""
    SELECT
        c.firstname || ' ' || c.lastname AS Name,
        ROUND(SUM((p.productunitprice * o.quantityordered)::numeric), 2) AS Total
    FROM orderdetail o
    JOIN customer c ON o.customerid = c.customerid
    JOIN product  p ON o.productid  = p.productid
    WHERE (c.firstname || ' ' || c.lastname) = '{CustomerName}'
    GROUP BY c.customerid;
    """


def ex3(conn):
    return """
    SELECT
        c.firstname || ' ' || c.lastname AS Name,
        ROUND(SUM((p.productunitprice * o.quantityordered)::numeric), 2) AS Total
    FROM orderdetail o
    JOIN customer c ON o.customerid = c.customerid
    JOIN product  p ON o.productid  = p.productid
    GROUP BY c.customerid
    ORDER BY Total DESC;
    """


def ex4(conn):
    return """
    SELECT
        r.region AS Region,
        ROUND(SUM((p.productunitprice * o.quantityordered)::numeric), 2) AS Total
    FROM orderdetail o
    JOIN customer c ON o.customerid = c.customerid
    JOIN country  co ON c.countryid = co.countryid
    JOIN region   r  ON co.regionid = r.regionid
    JOIN product  p  ON o.productid = p.productid
    GROUP BY r.region
    ORDER BY Total DESC;
    """


def ex5(conn):
    return """
    SELECT
        co.country AS Country,
        ROUND(SUM((p.productunitprice * o.quantityordered)::numeric)) AS Total
    FROM orderdetail o
    JOIN customer c ON o.customerid = c.customerid
    JOIN country  co ON c.countryid = co.countryid
    JOIN product  p  ON o.productid = p.productid
    GROUP BY co.country
    ORDER BY Total DESC;
    """


def ex6(conn):
    return """
    SELECT
        r.region AS Region,
        co.country AS Country,
        ROUND(SUM((p.productunitprice * o.quantityordered)::numeric)) AS CountryTotal,
        RANK() OVER (
            PARTITION BY r.region
            ORDER BY SUM((p.productunitprice * o.quantityordered)::numeric) DESC
        ) AS TotalRank
    FROM orderdetail o
    JOIN customer c ON o.customerid = c.customerid
    JOIN country  co ON c.countryid = co.countryid
    JOIN region   r  ON co.regionid = r.regionid
    JOIN product  p  ON o.productid = p.productid
    GROUP BY r.region, co.country
    ORDER BY r.region ASC, TotalRank ASC;
    """


def ex7(conn):
    return """
    WITH RankedCountries AS (
        SELECT
            r.region AS region,
            co.country AS country,
            ROUND(SUM((p.productunitprice * o.quantityordered)::numeric)) AS CountryTotal,
            RANK() OVER (
                PARTITION BY r.region
                ORDER BY SUM((p.productunitprice * o.quantityordered)::numeric) DESC
            ) AS CountryRegionalRank
        FROM orderdetail o
        JOIN customer c ON o.customerid = c.customerid
        JOIN country  co ON c.countryid = co.countryid
        JOIN region   r  ON co.regionid = r.regionid
        JOIN product  p  ON o.productid = p.productid
        GROUP BY r.region, co.country
    )
    SELECT
        region AS Region,
        country AS Country,
        CountryTotal,
        CountryRegionalRank
    FROM RankedCountries
    WHERE CountryRegionalRank = 1
    ORDER BY region ASC, country ASC;
    """


def ex8(conn):
    return """
    WITH base AS (
        SELECT
            CASE
                WHEN EXTRACT(MONTH FROM o.orderdate) BETWEEN 1 AND 3 THEN 'Q1'
                WHEN EXTRACT(MONTH FROM o.orderdate) BETWEEN 4 AND 6 THEN 'Q2'
                WHEN EXTRACT(MONTH FROM o.orderdate) BETWEEN 7 AND 9 THEN 'Q3'
                ELSE 'Q4'
            END AS Quarter,
            EXTRACT(YEAR FROM o.orderdate)::INT AS Year,
            o.customerid AS CustomerID,
            (p.productunitprice * o.quantityordered)::numeric AS line_total
        FROM orderdetail o
        JOIN product p ON o.productid = p.productid
    )
    SELECT
        Quarter,
        Year,
        CustomerID,
        ROUND(SUM(line_total)) AS Total
    FROM base
    GROUP BY Quarter, Year, CustomerID
    ORDER BY Year, Quarter, CustomerID;
    """


def ex9(conn):
    return """
    WITH QuarterlySales AS (
        SELECT
            CASE
                WHEN EXTRACT(MONTH FROM o.orderdate) BETWEEN 1 AND 3 THEN 'Q1'
                WHEN EXTRACT(MONTH FROM o.orderdate) BETWEEN 4 AND 6 THEN 'Q2'
                WHEN EXTRACT(MONTH FROM o.orderdate) BETWEEN 7 AND 9 THEN 'Q3'
                ELSE 'Q4'
            END AS Quarter,
            EXTRACT(YEAR FROM o.orderdate)::INT AS Year,
            o.customerid AS CustomerID,
            ROUND(SUM((p.productunitprice * o.quantityordered)::numeric)) AS Total
        FROM orderdetail o
        JOIN product p ON o.productid = p.productid
        GROUP BY
            CASE
                WHEN EXTRACT(MONTH FROM o.orderdate) BETWEEN 1 AND 3 THEN 'Q1'
                WHEN EXTRACT(MONTH FROM o.orderdate) BETWEEN 4 AND 6 THEN 'Q2'
                WHEN EXTRACT(MONTH FROM o.orderdate) BETWEEN 7 AND 9 THEN 'Q3'
                ELSE 'Q4'
            END,
            EXTRACT(YEAR FROM o.orderdate)::INT,
            o.customerid
    ),
    RankedSales AS (
        SELECT
            Quarter,
            Year,
            CustomerID,
            Total,
            RANK() OVER (
                PARTITION BY Year, Quarter
                ORDER BY Total DESC
            ) AS CustomerRank
        FROM QuarterlySales
    )
    SELECT
        Quarter,
        Year,
        CustomerID,
        Total,
        CustomerRank
    FROM RankedSales
    WHERE CustomerRank <= 5
    ORDER BY Year, Quarter, Total DESC;
    """


def ex10(conn):
    return """
    WITH MonthlyTotals AS (
        SELECT
            EXTRACT(MONTH FROM o.orderdate)::INT AS MonthNum,
            ROUND(SUM((p.productunitprice * o.quantityordered)::numeric)) AS Total
        FROM orderdetail o
        JOIN product p ON o.productid = p.productid
        GROUP BY EXTRACT(MONTH FROM o.orderdate)::INT
    )
    SELECT
        TO_CHAR(TO_DATE(MonthNum::TEXT, 'MM'), 'Month') AS Month,
        Total,
        RANK() OVER (ORDER BY Total DESC) AS TotalRank
    FROM MonthlyTotals
    ORDER BY TotalRank;
    """


def ex11(conn):
    return """
    WITH OrderedDates AS (
        SELECT DISTINCT
            c.customerid,
            c.firstname,
            c.lastname,
            co.country,
            o.orderdate
        FROM orderdetail o
        JOIN customer c ON o.customerid = c.customerid
        JOIN country  co ON c.countryid = co.countryid
    ),
    LaggedDates AS (
        SELECT
            customerid,
            firstname,
            lastname,
            country,
            orderdate,
            LAG(orderdate, 1) OVER (
                PARTITION BY customerid
                ORDER BY orderdate
            ) AS PreviousOrderDate
        FROM OrderedDates
    ),
    Diffs AS (
        SELECT
            customerid,
            firstname,
            lastname,
            country,
            orderdate,
            PreviousOrderDate,
            (orderdate - PreviousOrderDate) AS MaxDaysWithoutOrder
        FROM LaggedDates
        WHERE PreviousOrderDate IS NOT NULL
    ),
    RankedDiffs AS (
        SELECT
            *,
            ROW_NUMBER() OVER (
                PARTITION BY CustomerID
                ORDER BY MaxDaysWithoutOrder DESC, OrderDate ASC
            ) AS rn
        FROM Diffs
    )
    SELECT
        CustomerID AS CustomerID,
        FirstName AS FirstName,
        LastName AS LastName,
        Country AS Country,
        OrderDate AS OrderDate,
        PreviousOrderDate AS PreviousOrderDate,
        MaxDaysWithoutOrder AS MaxDaysWithoutOrder
    FROM RankedDiffs
    WHERE rn = 1
    ORDER BY MaxDaysWithoutOrder DESC, CustomerID DESC;
    """


## Running Example Queries (ex3, ex4, ex6, ex7, ex8, ex10, ex11)

Below we execute several of the `ex` functions against the PostgreSQL
database and display the results using pandas.

In [11]:
def run_query(sql: str):
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn)


In [ ]:
df3 = run_query(ex3(None))   # conn param ignored
df3.head()

,name,total
0,Annette Roulet,6092806.79
1,Manuel Pereira,5905784.45
2,Janete Limeira,5817785.32
3,Christina Berglund,5679042.68
4,Liz Nixon,5623333.51


In [13]:
df4 = run_query(ex4(None))
df4


,region,total
0,Western Europe,1.399034e+08
1,South America,8.367663e+07
2,North America,8.019435e+07
3,Southern Europe,4.879062e+07
4,British Isles,3.899529e+07
5,Central America,2.514686e+07
6,Northern Europe,2.109077e+07
7,Scandinavia,1.500917e+07
8,Eastern Europe,5.196640e+06


In [14]:
df10 = run_query(ex10(None))
df10


,month,total,totalrank
0,December,43729002.0,1
1,January,43700347.0,2
2,November,42467339.0,3
3,August,42086341.0,4
4,October,41822571.0,5
5,September,41260045.0,6
6,July,39415220.0,7
7,February,35723879.0,8
8,May,33537970.0,9
9,April,32109078.0,10


In [18]:
df11 = run_query(ex11(None))
df11


,customerid,firstname,lastname,country,orderdate,previousorderdate,maxdayswithoutorder
0,70,Patricio,Simpson,Argentina,2015-03-20,2015-01-01,78
1,26,Frederique,Citeaux,France,2014-05-02,2014-02-23,68
2,54,Manuel,Pereira,Venezuela,2015-04-24,2015-02-18,65
3,8,Antonio,Moreno,Mexico,2015-06-16,2015-04-14,63
4,23,Felipe,Izquierdo,Venezuela,2014-12-16,2014-10-16,61
...,...,...,...,...,...,...,...
86,51,Liu,Wong,USA,2014-11-05,2014-10-06,30
87,64,Michael,Holz,Switzerland,2015-06-16,2015-05-18,29
88,41,Jean,Fresniere,Canada,2016-01-24,2015-12-27,28
89,28,Giovanni,Rovelli,Italy,2012-12-02,2012-11-04,28
